In [1]:
# first step is to download and extract the .rar files from https://github.com/lasigeBioTM/IHP
# this code assumes each .rar file is saved in a folder in the same directory as the code
# only input needed is the name of the folder for the .rar file

In [2]:
# import dependencies

import pandas as pd
import os
import re

In [5]:
# define functions

def read_in_data(path):
    data = []
    files = [f for f in os.listdir(path)]
    for f in files:
        with open (os.path.join(path,f), "r") as myfile:
            data.append(myfile.read())
    return data, files

def extract_all(regex, x, numeric):
    all_items = x.split('\n')[:-1]
    out_list = []
    for item in all_items:
        if numeric == False:
            out_list.append(re.search(regex,item).group(1))
        else:
            out_list.append(int(re.search(regex,item).group(1))) 
    return out_list

def expand_labels(df):
    df['HPO_symptoms'] = df['labels'].apply(lambda x: extract_all('\| (.*)',x,False))
    df['HPO_codes'] = df['labels'].apply(lambda x: extract_all('\tHP_(.*) \|',x,False))
    df['starts'] = df['labels'].apply(lambda x: extract_all('\[(\d*)::',x,True))
    df['ends'] = df['labels'].apply(lambda x: extract_all('::(\d*)\]',x,True))
    df.drop(columns='labels',inplace=True)
    
def output_GSC_data(GSC_folder):
    
    # read in data as a dataframe
    text_data, text_files = read_in_data(os.path.join(os.getcwd(),GSC_folder+'\\Text'))
    annot_data, annot_files = read_in_data(os.path.join(os.getcwd(),GSC_folder+'\\Annotations'))
    print('Filenames match:',text_files==annot_files)
    if text_files==annot_files:
        df = pd.DataFrame({'file':text_files, 'text':text_data, 'labels':annot_data})

        # parse data
        expand_labels(df)

        # output as csv
        filename = GSC_folder + '.csv'
        df.to_csv(filename, index=False)
    else:
        raise Exception('Need to match filenames before constructing datasets')

In [6]:
# get data

output_GSC_data('GSC_v2')
output_GSC_data('GSC+')

Filenames match: True
Filenames match: True
